# SQL project - Languages 

Please, be aware that this SQL code is written in Python environment. 
_To access each table, one need to specify the schema_name.table_name in the queries (e.g., `world.countries` for the `countries` table, and `world.languages` for the `languages` table)._

### Notice to the world database

Please, be aware that the world db is uncomplete and might (and does) have some errors.
Hence, the results received might have a little in common with an actual state.

**Country names** are in _countries_plus_ table (not in _countries_ table), and yet not all of country names are there.

# MOST POPULAR LANGUAGES in the world

In [5]:
-- How many languages people speak in the world (according to the world database)?

SELECT COUNT(DISTINCT world.languages.name) AS no_of_lang
FROM world.languages;

,no_of_lang
0,396


According to the world database, there is **396 unique languages** in the world.


In [6]:
-- Which languages (show top 10) are spoken in most of the countries of the world? 
-- In how many countries are they considered as official and unofficial languages? 
-- solution 1: COUNT with CASE WHEN (classification)

SELECT 
    world.languages.name AS language, 
    COUNT(world.languages.name) AS no_of_countries,
    COUNT(CASE WHEN official ='True' THEN world.languages.name END) AS as_official,
	COUNT(CASE WHEN official ='False' THEN world.languages.name END) AS as_unofficial
FROM world.languages
WHERE world.languages.name <> 'unspecified' AND world.languages.name <> 'Other'   -- removed from the result set
GROUP BY world.languages.name
ORDER BY no_of_countries DESC, world.languages.name
LIMIT 10;

-- Please, have a look at the chart.

,language,no_of_countries,as_official,as_unofficial
0,English,104,58,46
1,French,49,32,17
2,Arabic,32,24,8
3,Spanish,28,18,10
4,Russian,18,4,14
5,German,13,6,7
6,Italian,12,3,9
7,Portuguese,11,6,5
8,Chinese,9,0,9
9,Armenian,6,2,4


,language,no_of_countries,as_official,as_unofficial
0,English,104,58,46
1,French,49,32,17
2,Arabic,32,24,8
3,Spanish,28,18,10
4,Russian,18,4,14
5,German,13,6,7
6,Italian,12,3,9
7,Portuguese,11,6,5
8,Chinese,9,0,9
9,Armenian,6,2,4


**Conclusion:** 
according to the world database, the **top 3 languages**, spoken **in the highest number of countries** are English, French and Arabic. 
The same order applies when it comes to **official** languages, but in the ranking of the top 3 **unofficial** languages Russian takes the 3rd place (instead of Arabic).


In [8]:
-- Which languages (show top 10) are spoken in most of the countries of the world? 
-- In how many countries are they considered as official and unofficial languages? 
-- solution 2: correlated subquery

SELECT 
    world.languages.name AS language, 
    COUNT(world.languages.name) AS no_of_countries,
    (SELECT COUNT(l2.name) FROM world.languages l2 WHERE official ='True' AND world.languages.name =l2.name) AS as_official,        -- correlated subquery
    (SELECT COUNT(l3.name) FROM world.languages l3 WHERE official ='False' AND world.languages.name =l3.name) AS as_unofficial
FROM world.languages
WHERE world.languages.name <> 'unspecified' AND world.languages.name <> 'Other'   -- removed from the result set
GROUP BY world.languages.name
ORDER BY no_of_countries DESC, world.languages.name
LIMIT 10;

-- Please, have a look at the chart.

,language,no_of_countries,as_official,as_unofficial
0,English,104,58,46
1,French,49,32,17
2,Arabic,32,24,8
3,Spanish,28,18,10
4,Russian,18,4,14
5,German,13,6,7
6,Italian,12,3,9
7,Portuguese,11,6,5
8,Chinese,9,0,9
9,Armenian,6,2,4


In [9]:
-- TOP 10 countries with the highest number of languages spoken.

SELECT cp.name AS country,
        COUNT(l.lang_id) AS no_languages
FROM world.languages l
JOIN world.countries_plus cp
ON l.code = cp.code
GROUP BY cp.name
ORDER BY no_languages DESC
LIMIT 10;  

-- Please, have a look at the chart.


,country,no_languages
0,Zambia,19
1,Zimbabwe,16
2,Ethiopia,16
3,Nepal,14
4,India,14
5,France,13
6,South Africa,13
7,Mali,13
8,Angola,12
9,Malawi,12


**Conclusion:** African countries and India & Nepal are the **regions with the highest language diversity** in the world. Up to 19 languages in one country (Zambia).

In [24]:
/* Countries where people speak 3 or 2 of the top languages - English, French and/ or Arabic. 
Note: There is more countries (codes) in languages table than in countries_plus table, i.e. AIA is Anguilla but there is no name for it in countries_plus table. */

WITH ENG AS  -- CTE
(SELECT l.code AS country_code,
 cp.name AS countries_Eng
FROM world.languages l
LEFT JOIN world.countries_plus cp
ON l.code= cp.code
WHERE l.name = 'English'
),

FR AS
(SELECT l.code AS country_code,
 cp.name AS countries_Fr
FROM world.languages l
LEFT JOIN world.countries_plus cp
ON l.code= cp.code
WHERE l.name = 'French'
),

ARAB AS
(SELECT l.code AS country_code,
 cp.name AS countries_Arab
FROM world.languages l
LEFT JOIN world.countries_plus cp
ON l.code= cp.code
WHERE l.name = 'Arabic'
)

SELECT countries_Eng,
       countries_Fr,
       countries_Arab
FROM world.countries_plus cp
FULL JOIN ENG
ON cp.code = ENG.country_code
FULL JOIN FR
ON cp.code = FR.country_code
FULL JOIN ARAB 
ON cp.code = ARAB.country_code 
WHERE NOT (ENG.country_code <> FR.country_code AND ENG.country_code <>ARAB.country_code AND FR.country_code <> ARAB.country_code)   -- removed countries where only 1 of top languages is spoken 
ORDER BY
(CASE WHEN countries_Eng = countries_Fr AND countries_Eng = countries_Arab THEN 1
     WHEN countries_Eng = countries_Fr THEN 2
     WHEN countries_Eng = countries_Arab THEN 3
     WHEN countries_Fr = countries_Arab THEN 4
    ELSE 5 END),
    countries_Eng,
    countries_Fr;
    

,countries_eng,countries_fr,countries_arab
0,Canada,Canada,Canada
1,Lebanon,Lebanon,Lebanon
2,Panama,Panama,Panama
3,Syria,Syria,Syria
4,Argentina,Argentina,None
5,Burundi,Burundi,None
6,Cameroon,Cameroon,None
7,Hungary,Hungary,None
8,Laos,Laos,None
9,Madagascar,Madagascar,None


**Conclusion:** 
There are **four countries** where people speak **English**, **French**, as well as **Arabic**, that is in Canada, Lebanon, Panama and Syria.

In [11]:
-- Top 10 languages spoken by the most of the people in the world.
-- Please be aware that the results might be underreported as in language table for some countries there is lack of information concerning the percentage of population speaking a given language (i.e. Germany - DEU - percentage IS NULL, China - CHN - percentage IS NULL).

WITH lang_country_people AS  -- number of people speaking a given language in a given country
(SELECT l.name AS language, 
         l.code AS country_code, 
     ROUND((l.percent/100) * p.size) AS no_people  -- rounded to integer
 FROM world.languages l
 JOIN world.populations p
 ON l.code = p.country_code
 WHERE l.name<>'Other' AND l.name<>'unspecified' AND l.name<> 'unknown' AND p.year =2015
)

SELECT lcp.language, 
        SUM(no_people) AS total_people,
        SUM(no_people)/1000000 AS total_people_M
FROM lang_country_people lcp
WHERE lcp.no_people IS NOT NULL
GROUP BY lcp.language
ORDER BY total_people DESC, lcp.language
LIMIT 10;

-- Please, have a look at the chart.

,language,total_people,total_people_m
0,Hindi,537604234,537.604234
1,English,313666829,313.666829
2,Spanish,242019708,242.019708
3,Russian,167783048,167.783048
4,Bangla,159063705,159.063705
5,Punjabi,127895236,127.895236
6,Bengali,106195095,106.195095
7,Telugu,94395633,94.395633
8,Marathi,91773535,91.773535
9,Urdu,81407871,81.407871


**Conclusion:**

Although **English, French** and **Arabic** are the languages spoken in **the highest number of countries,** 
when it comes to the **number of people** who are able to speak in a given language then the top 3 are: **Hindi** (over 537M od people), **English** (over 313M) and **Spanish** (over 242M).

_**Note**: Please be aware that the results might be underreported as in language table for some countries there is lack of information concerning the percentage of population speaking a given language (i.e. **Germany** - DEU - percentage IS NULL, **China** - CHN - percentage IS NULL)._



In [12]:
-- Top 5 languages spoken by the most of the people - per continent.
-- Note: Antarctica is excluded, there are only 6 continents in the output.
-- Moreover, please be aware that the results might be underreported as in language table for some countries there is lack of information concerning the percentage of population speaking a given language (i.e. Germany - DEU - percentage IS NULL, China - CHN - percentage IS NULL).

WITH lang_country_people AS  -- number of people speaking a given language in a given country
(SELECT l.name AS language, 
         l.code AS country_code, 
     ROUND((l.percent/100) * p.size) AS no_people  -- rounded to integer
 FROM world.languages l
 JOIN world.populations p
 ON l.code = p.country_code
 WHERE l.name<>'Other' AND l.name<>'unspecified' AND l.name<> 'unknown' AND p.year =2015
),
continent_lang_pp AS   -- number of people speaking a given language in a given continent
(SELECT cp.continent, 
         lcp.language, 
        SUM(no_people)/1000000 AS total_people_M,
         RANK() OVER(PARTITION BY cp.continent ORDER BY (SUM(no_people)/1000000) DESC) AS rank
FROM lang_country_people lcp
JOIN world.countries_plus cp
ON lcp.country_code = cp.code
WHERE lcp.no_people IS NOT NULL
GROUP BY cp.continent, lcp.language
)

SELECT continent, language, total_people_M AS "total number of people speaking that language (in M)"
FROM continent_lang_pp
WHERE rank BETWEEN 1 AND 10
ORDER BY continent, total_people_M DESC, language;


,continent,language,total number of people speaking that language (in M)
0,Africa,Oromo,33.594073
1,Africa,Amharic,29.121490
2,Africa,Portuguese,20.996574
3,Africa,Sudanic family,16.295013
4,Africa,IsiZulu,12.487719
5,Africa,Kinyarwanda,11.540008
6,Africa,IsiXhosa,8.801916
7,Africa,Bambara,8.148658
8,Africa,Afrikaans,7.682335
9,Africa,Emakhuwa,7.078399


**Conclusion:** 
According to the world database, 
- in **Africa** the most people speak Oromo, Amharic and Portuguese,
- in **Asia** these are Hindi,Bangla and Punjabi,
- in **Europe** - Russian (although most of Russia is located in Asia), French and Polish,
- in **North America** - mostly English and Spanish and then French,
- in **Oceania** - mostly English and then Mandarin and Italian,
- in **South America** - mostly Spanish and then Quechua and Aymara.

_**Note**: Please be aware that the results might be underreported as in language table for some countries there is lack of information concerning the percentage of population speaking a given language (i.e. **Germany** - DEU - percentage IS NULL, **China** - CHN - percentage IS NULL)._

### - - - _Side notes:_ - - -

In [1]:
-- Side notes: checking the data available in the populations table

-- checking which year has more data that can be used to the calculation
SELECT  p.country_code, COUNT(p.year) AS no
FROM world.populations p
GROUP BY p.country_code
HAVING COUNT(p.year) <2 OR COUNT(p.year)>2 -- Out: 0 rows, so all countries have data for 2 years

UNION ALL
SELECT p.country_code, p.year
FROM world.populations p
WHERE p.year NOT IN (2010,2015);  -- Out: 0 rows, so there are data only for these 2 years: 2010 and 2015

,country_code,no


In [14]:
WITH lang_country_people AS  -- number of people speaking a given language in a given country
(SELECT l.name AS language, 
         l.code AS country_code, 
     ROUND((l.percent/100) * p.size) AS no_people  -- rounded to integer
 FROM world.languages l
 JOIN world.populations p
 ON l.code = p.country_code
 WHERE l.name<>'Other' AND l.name<>'unspecified' AND l.name<> 'unknown' AND p.year =2015
)

SELECT * 
FROM lang_country_people
WHERE language = 'German';

,language,country_code,no_people
0,German,ARG,NaN
1,German,AUT,7653592.0
2,German,BEL,112494.0
3,German,BLZ,11497.0
4,German,CAN,466032.0
5,German,DNK,NaN
6,German,DEU,NaN
7,German,HUN,1102419.0
8,German,ISL,NaN
9,German,ITA,NaN


In [15]:
SELECT * FROM world.populations
WHERE country_code= 'DEU';

,pop_id,country_code,year,fertility_rate,life_expectancy,size
0,145,DEU,2010,1.39,79.98781,81776930
1,144,DEU,2015,1.50,81.09024,81679770


In [16]:
-- checking the data available in the database

SELECT * FROM world.languages
WHERE code= 'DEU' OR code= 'ESP' OR code='CHN'
ORDER BY code;

,lang_id,code,name,percent,official
0,179,CHN,Mandarin,NaN,True
1,180,CHN,Yue,NaN,False
2,181,CHN,Wu,NaN,False
3,182,CHN,Minbei,NaN,False
4,183,CHN,Minnan,NaN,False
5,184,CHN,Xiang,NaN,False
6,185,CHN,Gan,NaN,False
7,186,CHN,Hakka,NaN,False
8,187,CHN,Other,NaN,False
9,317,DEU,German,NaN,True


In [17]:
SELECT * FROM world.languages
WHERE name='Polish';

,lang_id,code,name,percent,official
0,466,LTU,Polish,5.6,False
1,692,POL,Polish,98.2,True


# COUNTRIES

In [19]:
-- The country with a biggest surface area in each of continents.

WITH max_area AS
(SELECT c.continent, MAX(c.surface_area) AS max_sa
FROM world.countries c
GROUP BY c.continent)

SELECT c.continent, c.name AS country, m.max_sa AS surface_area
FROM world.countries c
JOIN max_area m
ON c.surface_area = m.max_sa AND c.continent = m.continent
ORDER BY c.surface_area DESC;


,continent,country,surface_area
0,Europe,Russian Federation,17075400
1,North America,Canada,9970610
2,Asia,China,9572900
3,South America,Brazil,8547400
4,Oceania,Australia,7741220
5,Africa,Sudan,2505810


## Number of presidents and PMs

In [18]:
-- for each country check the number of presidents and prime ministers in the database.

SELECT 
    CASE WHEN world.presidents.country IS NULL THEN world.prime_ministers.country
 	WHEN world.prime_ministers.country IS NULL THEN world.presidents.country
    ELSE world.presidents.country
    END AS country,
    COUNT(president) as presidents_no, 
    COUNT(prime_minister) as PM_no
FROM world.presidents
FULL JOIN world.prime_ministers
ON world.presidents.country = world.prime_ministers.country
GROUP BY world.presidents.country, world.prime_ministers.country
ORDER BY presidents_no DESC, PM_no DESC, country;



,country,presidents_no,pm_no
0,Egypt,1,1
1,Haiti,1,1
2,Portugal,1,1
3,Vietnam,1,1
4,Chile,1,0
5,Liberia,1,0
6,Uruguay,1,0
7,Australia,0,1
8,Brunei,0,1
9,India,0,1


**Conclusion:** It looks like there is only 1 name of president/ PM given for each country. No historical data available.